In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
df = pd.read_csv("/Users/elijahwoolford/Desktop/results.csv")
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [4]:
def getWinningTeam(net_score, home_team, away_team):
    team = ""
    if net_score > 0:
        team = home_team
    elif net_score < 0:
        team = away_team
    else:
        team = "Draw"
    return team

In [6]:
df["year"] = df["date"].apply(lambda x: x[:4])
df["net_score"] = df["home_score"] - df["away_score"]
df["winning_team"] = df.apply(lambda row: getWinningTeam(row["net_score"], row["home_team"], row["away_team"]), axis=1)

In [24]:
df.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,net_score,winning_team,year
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,Draw,1872
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2,England,1873
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,Scotland,1874
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0,Draw,1875
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3,Scotland,1876
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False,4,Scotland,1876
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False,-2,Scotland,1877
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False,-2,Scotland,1877
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False,5,Scotland,1878
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False,9,Scotland,1878


In [45]:
X = df[["home_score", "away_score", "neutral", "net_score", "year"]]

tournament = pd.get_dummies(df["tournament"])
country = pd.get_dummies(df["country"])
home_team = pd.get_dummies(df["home_team"])
away_team = pd.get_dummies(df["away_team"])

X = pd.concat([X, tournament, country, home_team, away_team], axis=1)
y = df["winning_team"]

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

In [32]:
mlp = MLPClassifier(hidden_layer_sizes=(10,))
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [33]:
mlp.score(X_test, y_test)

0.27417727903164796

In [34]:
cross_val_score(mlp, X_test, y_test, cv=5)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.40487062, 0.30223187, 0.24976348, 0.23274478, 0.30244698])

In [47]:
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [48]:
clf.score(X_test, y_test)

0.3062665489849956

In [49]:
cross_val_score(clf, X_test, y_test, cv=10)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.28099652, 0.28648005, 0.28961749, 0.29375387, 0.30333543,
       0.28087012, 0.33116883, 0.30592105, 0.31266667, 0.33963538])